In [1]:
!pip install nltk

                                              0.0/1.5 MB ? eta -:--:--
                                              0.0/1.5 MB ? eta -:--:--
                                              0.0/1.5 MB ? eta -:--:--
                                              0.0/1.5 MB 262.6 kB/s eta 0:00:06
                                              0.0/1.5 MB 262.6 kB/s eta 0:00:06
                                              0.0/1.5 MB 262.6 kB/s eta 0:00:06
                                              0.0/1.5 MB 262.6 kB/s eta 0:00:06
     -                                        0.0/1.5 MB 122.9 kB/s eta 0:00:12
     -                                        0.0/1.5 MB 122.9 kB/s eta 0:00:12
     -                                        0.0/1.5 MB 122.9 kB/s eta 0:00:12
     -                                        0.0/1.5 MB 122.9 kB/s eta 0:00:12
     -                                        0.0/1.5 MB 122.9 kB/s eta 0:00:12
     -                                        0.0/1.5 MB 122.9 kB/s


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: C:\Users\Onelity-krazgkelis\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [69]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Onelity-
[nltk_data]     krazgkelis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Onelity-
[nltk_data]     krazgkelis\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to C:\Users\Onelity-
[nltk_data]     krazgkelis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
class JobSimilarityAnalyzer:
    def __init__(self):
        self.tokenizer = RegexpTokenizer(r'\w+')
        self.stop_words = set(stopwords.words('english'))

    def tokenize_and_filter(self, text):
        tokens = self.tokenizer.tokenize(text)
        filtered_tokens = [word.lower() for word in tokens if word.lower() not in self.stop_words]

        pos_tags = pos_tag(filtered_tokens)
        filtered_words = [word for word, pos in pos_tags if pos in ['NNP', 'NN']]

        if not len(filtered_words) > 1:
            raise ValueError("The length is wrong")

        return ' '.join(filtered_words)

    def get_cosine_similarity(self, text1, text2):
        vectorizer = CountVectorizer()
        matrix = vectorizer.fit_transform([text1, text2])
        similarity = cosine_similarity(matrix)
        return similarity[0][1]

    def process_job_files(self, file_paths):
        job_descriptions = []

        for file_path in file_paths:
            with open("/content/drive/MyDrive/Colab Notebooks/jobs/" + file_path, 'r') as file:
                text = file.read()
                filtered_tokens = self.tokenize_and_filter(text)
                job_descriptions.append([file_path, filtered_tokens])

        return job_descriptions

    def calculate_similarity_scores(self, job_descriptions, user_text):
        user_text_tokens = self.tokenize_and_filter(user_text)
        similarity_scores = []

        for job in job_descriptions:
            similarity = self.get_cosine_similarity(user_text_tokens, job[1])
            similarity_scores.append((job[0], similarity))

        similarity_scores.sort(key=lambda x: x[1], reverse=True)

        return similarity_scores


In [79]:

test_sentence = """As a highly skilled and motivated data engineering professional, I possess a robust foundation in database design, having successfully implemented and optimized scalable database solutions using technologies such as SQL, NoSQL, and cloud-based data storage systems. My proficiency extends to ETL (Extract, Transform, Load) processes, where I have demonstrated expertise in designing and implementing efficient data pipelines to seamlessly integrate and transform diverse data sources, ensuring data quality and integrity.

I am adept at utilizing programming languages such as Python and Java to develop custom data processing scripts and applications, automating data workflows, and enhancing overall system efficiency. My experience includes working with big data technologies, including Hadoop and Spark, enabling me to process and analyze large datasets with a focus on performance optimization.

In addition, I have a strong understanding of data warehousing concepts and have effectively designed and maintained data warehouses, providing stakeholders with timely and accurate business intelligence. My experience with data modeling and schema design contributes to creating well-organized and accessible data structures that cater to specific business requirements.

Furthermore, I am well-versed in cloud platforms such as AWS, Azure, and Google Cloud, utilizing their services for data storage, processing, and analytics. My expertise extends to version control systems, ensuring proper documentation and collaborative development practices. As a proactive problem solver, I am accustomed to troubleshooting and resolving complex data issues, ensuring the continuous flow and availability of high-quality data for analytical purposes.

With a keen eye for emerging trends in data engineering and a commitment to staying abreast of technological advancements, I am confident in my ability to contribute effectively to any data engineering team, driving innovation and delivering robust solutions to meet the dynamic needs of the organization."""


analyzer = JobSimilarityAnalyzer()

file_paths = [ "data_engineer.txt", "backend_developer.txt", "frontend_developer.txt", "sales_manager.txt"]

# Process job files and calculate similarity scores
job_descriptions = analyzer.process_job_files(file_paths)
similarity_scores = analyzer.calculate_similarity_scores(job_descriptions, test_sentence)

# Print similarity scores
print("You most likely want to do this jobs: ")
for idx1, similarity in similarity_scores:
    print(f"{idx1}: {similarity}")

You most likely want to do this jobs: 
data_engineer.txt: 0.3828478769486491
frontend_developer.txt: 0.27183741892777974
backend_developer.txt: 0.18072289156626506
sales_manager.txt: 0.14065447086831775


In [73]:
def tokenize_and_filter(text, stop_words=None):
    if stop_words is None:
        stop_words = set(stopwords.words('english'))

    # Assuming you have a tokenizer initialized somewhere, replace with your own tokenizer
    tokenizer = RegexpTokenizer(r'\w+')  # Replace with your tokenizer

    tokens = tokenizer.tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words]

    pos_tags = pos_tag(filtered_tokens)
    filtered_words = [word for word, pos in pos_tags if pos in ['NNP', 'NN']]

    if not len(filtered_words) > 1:
        return None
    return ' '.join(filtered_words)


def get_cosine_similarity(text1, text2):
    vectorizer = CountVectorizer()
    matrix = vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(matrix)
    return similarity[0][1]

analyzer = JobSimilarityAnalyzer()

# read the linked in dataset with the jobs
df = pd.read_csv("datasets/job_postings.csv")

# start cleaning the title to remove the irellevent information
df['title'] = df['title'].apply(tokenize_and_filter)

#retain only the job titles that have more than 5 posts
result_df = df[df.groupby('title')['description'].transform('count') > 5].groupby('title')['description'].agg(lambda x: ' '.join(str(desc) for desc in x if pd.notna(desc))).reset_index()

# clean the desctiptions
result_df["description"] = result_df["description"].apply(tokenize_and_filter)


In [74]:
# the question we want to make

test_sentence = """About me
Cloud Expert | Developer | DevOps | Cloud Architect Machine Learning | AI | LLM ★ AWS (Amazon Web
Services) 4X Certified ★ 11 Years of Experience across AWS, Azure and Google Cloud Platform ★
Designing, Automation and Deploying web applications at scale ★ Experience with Fortune... Read more
Skills

DevOps
Docker
AWS
Google Cloud Platform
Microsoft Azure
Kubernetes
Python
Django
Software architecture
Infrastructure management
Linux"""

result_df["similarity"] = result_df.apply(lambda row: get_cosine_similarity(row['description'], tokenize_and_filter(test_sentence)), axis=1)
result_df = result_df.sort_values(by='similarity', ascending=False).reset_index(drop=True)

result_df

,title,description,similarity
0,cloud architect,overview nformation security security security...,0.389613
1,azure engineer,interwell health kidney care management compan...,0.376700
2,infrastructure engineer,grayson search cybersecurity infrastructure fi...,0.325275
3,python developer,ecs python developer customer work memphis tn ...,0.320881
4,dotnet developer,hi ranjith advent position developer client cl...,0.305967
...,...,...,...
464,entry agent work home,fill entry position entry level role work pdf ...,0.000000
465,block tax advisor,bring team year round client relationshipsprov...,0.000000
466,package part time,job ground part warehouse transportation secto...,0.000000
467,pcu stepdown rn,impact place fusion staffing goal everyone tou...,0.000000
